In [1]:
import pandas as pd
import numpy as np
import os
import forzawrapper

In [2]:
def get_sign(score):
    if type(score) != str:
        return None
    h, a = int(score[0]), int(score[2])
    if h == a:
        return "X"
    elif h > a:
        return "1"
    else:
        return "2"

files = ["sheets/"+f for f in os.listdir("sheets")]

names = []
preds = []
for f in files:
    names.append('-'.join(f.split("-")[2:]).split(".")[0])
    tmp = pd.read_excel(f)
    preds.append(tmp[tmp.columns[5]][11:59].values.astype(str))

forza_df = forzawrapper.get_scores().iloc[:len(preds[0])]

pred_df = pd.DataFrame(np.array(preds).T, columns=names)
df = pd.concat([pred_df, forza_df], axis=1)
df["Correct"] = df["Score"].apply(lambda x: get_sign(x))
df["Total Correct Predictions"] = df.apply(lambda x: sum([int(v == x["Correct"]) for v in x[df.columns[:4]]]), axis=1)
df

,Irannejad-Armin,Filip,Martin-Hunhammar,Sellberg,Kickoff,Teams,Score,Correct,Total Correct Predictions
0,2,2,X,2,2022-11-20T16:00:00.000Z,Qatar vs Ecuador,0-2,2,3
1,X,1,1,1,2022-11-21T13:00:00.000Z,England vs Iran,6-2,1,3
2,2,2,2,2,2022-11-21T16:00:00.000Z,Senegal vs Netherlands,0-2,2,4
3,1,X,2,2,2022-11-21T19:00:00.000Z,USA vs Wales,1-1,X,1
4,1,1,1,1,2022-11-22T10:00:00.000Z,Argentina vs Saudi Arabia,1-2,2,0
5,1,1,1,1,2022-11-22T13:00:00.000Z,Denmark vs Tunisia,0-0,X,0
6,1,2,X,2,2022-11-22T16:00:00.000Z,Mexico vs Poland,0-0,X,1
7,1,1,1,1,2022-11-22T19:00:00.000Z,France vs Australia,4-1,1,4
8,2,1,2,2,2022-11-23T10:00:00.000Z,Morocco vs Croatia,0-0,X,0
9,1,1,1,1,2022-11-23T13:00:00.000Z,Germany vs Japan,1-2,2,0


In [8]:
print("Antal poäng efter", (~df['Correct'].isna()).sum(), "matcher")
names = []
scores = []
percent = []
points = []
for c in df.columns[:4]:
    names.append(c)
    scores.append((df[c] == df['Correct']).sum())
    percent.append(str(((df[c] == df['Correct']).sum()/(~df['Correct'].isna()).sum())*100)+"%")
    # Om nån pallar så får de gärna kolla/tänka igenom om poängberäkningen här blir rätt
    points.append(((df[c] == df['Correct']).astype(int)*(((len(files)*2-df["Total Correct Predictions"])+1)/2)).sum())
    
table = pd.DataFrame([names, points, scores, percent]).T
table.columns = ["Namn", "Poäng", "Antal Rätt", "Procent"]
table.sort_values("Poäng", ascending=False).reset_index(drop=True)

Antal poäng efter 19 matcher


,Namn,Poäng,Antal Rätt,Procent
0,Martin-Hunhammar,33.0,11,57.89473684210527%
1,Filip,28.0,10,52.63157894736842%
2,Sellberg,24.0,9,47.368421052631575%
3,Irannejad-Armin,22.0,8,42.10526315789473%
